# Convert image to Verilog code for controlling OLED

*** This Notebook is strictly for the use of Nguyen Van Binh and Nigel Ng. Ask the authors for any further use ***

*Better use offline with Jupyter Notebook so you don't need to upload the images. Just copy the code and ensure that you have all the libraries.*

# New Section

Import the libraries.
numpy, cv2, and PIL are compulsory. IPython is for displaying the image within the Notebook. sys is unneccessary.

In [1]:
import numpy as np
import cv2
import sys
from PIL import Image
from IPython import display

HEIGHT = 64
WIDTH = 96

If you run it with Jupyter Notebook on your laptop, just simply type in the route to the image file in filename. If you are using this notebook, upload the image and type in its name.

This will first resize the image (64x96), normalize the color data, and apply the filter (so there will be less data). If you want to see the image after being processed, uncomment the code snippet at the end of this block. The new image will be saved as "after.png"

In [2]:
filename = "win.png"
img_bgr = cv2.imread(filename)
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)


img = cv2.resize(img, (WIDTH, HEIGHT))
print(img.shape)


pixels = list()

for row in range(img.shape[0]):
    for col in range(img.shape[1]):
        # print(img[row][col])
        pixels.append(list(img[row][col]))

# normalize data to fixed bit
for pixel in pixels:
    pixel[0] = int(int(pixel[0] / 256 * 31) / 10) * 10
    pixel[1] = int(int(pixel[1] / 256 * 63) / 20) * 20
    pixel[2] = int(int(pixel[2] / 256 * 31) / 10) * 10
    
# for pixel in pixels:
#     pixel[0] = int(pixel[0] / 30 * 255)
#     pixel[1] = int(pixel[1] / 60 * 255)
#     pixel[2] = int(pixel[2] / 30 * 255)
 
    
# pixels = np.resize(pixels, (64, 96, 3))
# # print(pixels)
# array = np.array(pixels, dtype=np.uint8)
# new_image = Image.fromarray(array)

# new_image.save('after.png')
# new_image.show

(64, 96, 3)


Run the code below to have Verilog code at the output. Currently, it works best for image with dark background. The colors should also be high contrast. There will be updates for other background colors.

Copy the code to your stupid Verilog IDE to control the OLED, watch youtube while waiting for the bitstream being created, and feel like a pro after seeing the output.

In [3]:
# 5 red + 6 green + 5 blue
binary = dict()

for pixel, data in enumerate(pixels):
    if not (data[0] == 0 and data[1] == 0 and data[2] == 0):
        red = '{0:05b}'.format(data[0])
        green = '{0:06b}'.format(data[1])
        blue = '{0:05b}'.format(data[2])

        color = (red + green + blue)
        assert len(color) == 16
        if (color not in binary):
            binary[color] = [pixel]
        else:
            binary[color].append(pixel)   

def interval_extract(list):
    list = sorted(set(list))
    range_start = previous_number = list[0]
    
    for number in list[1:]:
        if number == previous_number + 1:
            previous_number = number
        else:
            yield [range_start, previous_number]
            range_start = previous_number = number
    yield [range_start, previous_number]
    
    
for i, color in enumerate(binary):
  if i != 0:
    print('else if (', end='')
    pixel_ranges = list(interval_extract(binary[color]))
    for i, pixel_range in enumerate(pixel_ranges):
        if i != len(pixel_ranges) - 1:
            if pixel_range[0] == pixel_range[1]:
                print('pixel_index == ' + str(pixel_range[0]) + ' || ', end='')
            else:
                print('((pixel_index >= ' + str(pixel_range[0]) + ') && (pixel_index <= ' + str(pixel_range[1]) + ")) || ", end='')
        else:
            if pixel_range[0] == pixel_range[1]:
                print('pixel_index == ' + str(pixel_range[0]) + ')', end='')
            else:
                print('(pixel_index >= ' + str(pixel_range[0]) + ') && (pixel_index <= ' + str(pixel_range[1]) + '))', end='')
            
    print(" oled_data = 16'b" + color + ';')
  else:
    print('if (', end='')
    pixel_ranges = list(interval_extract(binary[color]))
    for i, pixel_range in enumerate(pixel_ranges):
        if i != len(pixel_ranges) - 1:
            if pixel_range[0] == pixel_range[1]:
                print('pixel_index == ' + str(pixel_range[0]) + ' || ', end='')
            else:
                print('((pixel_index >= ' + str(pixel_range[0]) + ') && (pixel_index <= ' + str(pixel_range[1]) + ")) || ", end='')
        else:
            if pixel_range[0] == pixel_range[1]:
                print('pixel_index == ' + str(pixel_range[0]) + ')', end='')
            else:
                print('(pixel_index >= ' + str(pixel_range[0]) + ') && (pixel_index <= ' + str(pixel_range[1]) + '))', end='')
            
    print(" oled_data = 16'b" + color + ';')

print("else oled_data = 0;")


if (((pixel_index >= 2538) && (pixel_index <= 2539)) || pixel_index == 2732 || pixel_index == 2735 || pixel_index == 2826 || ((pixel_index >= 2919) && (pixel_index <= 2920)) || ((pixel_index >= 2923) && (pixel_index <= 2924)) || pixel_index == 2926 || pixel_index == 2930 || ((pixel_index >= 3012) && (pixel_index <= 3015)) || pixel_index == 3021 || pixel_index == 3026 || ((pixel_index >= 3108) && (pixel_index <= 3109)) || pixel_index == 3212 || pixel_index == 3315 || pixel_index == 3415 || pixel_index == 3498 || pixel_index == 3512 || pixel_index == 3591 || pixel_index == 3599 || pixel_index == 3686 || pixel_index == 3875 || pixel_index == 3883 || pixel_index == 3974 || pixel_index == 3979 || pixel_index == 3993 || ((pixel_index >= 4070) && (pixel_index <= 4071)) || ((pixel_index >= 4073) && (pixel_index <= 4075)) || pixel_index == 4163 || pixel_index == 4166 || ((pixel_index >= 4169) && (pixel_index <= 4171)) || pixel_index == 4262 || ((pixel_index >= 4265) && (pixel_index <= 4267)) ||

In [4]:
new = dict()

for i in binary:
  if (i != '0000000000000000'):
    new[i] = binary[i]

print(new)

{'0101000000000000': [2538, 2539, 2732, 2735, 2826, 2919, 2920, 2923, 2924, 2926, 2930, 3012, 3013, 3014, 3015, 3021, 3026, 3108, 3109, 3212, 3315, 3415, 3498, 3512, 3591, 3599, 3686, 3875, 3883, 3974, 3979, 3993, 4070, 4071, 4073, 4074, 4075, 4163, 4166, 4169, 4170, 4171, 4262, 4265, 4266, 4267, 4282, 4362, 4363, 4475, 4550, 4561, 4657, 4753, 4766, 4838, 4844, 4849, 4940, 4945, 5031, 5036, 5040, 5041, 5127, 5132, 5136, 5137, 5154, 5228, 5232, 5233, 5250, 5324, 5327, 5328, 5329, 5345, 5414, 5419, 5423, 5424, 5425, 5439, 5440, 5442, 5510, 5515, 5516, 5532, 5533, 5534, 5535, 5536], '1010000000000000': [2633], '1010001010001010': [2634, 2635, 2731, 2825, 2931, 3018, 3019, 3030, 3114, 3116, 3123, 3126, 3211, 3214, 3222, 3306, 3308, 3316, 3318, 3396, 3398, 3402, 3403, 3405, 3412, 3413, 3492, 3494, 3495, 3497, 3499, 3500, 3501, 3503, 3507, 3509, 3510, 3590, 3592, 3594, 3595, 3597, 3602, 3605, 3606, 3688, 3689, 3690, 3693, 3785, 3786, 3787, 3791, 3798, 3877, 3886, 3887, 3972, 3981, 3982, 3983

In [5]:
def coordinate(pixel):
  x = pixel % 96
  y = pixel // 96
  return (x,y)

pic = dict()

for color in new:
  pic[color] = list()
  for pixel in new[color]:
    pic[color].append(coordinate(pixel))

for color in pic:
  print(color + str(pic[color]))

0101000000000000[(42, 26), (43, 26), (44, 28), (47, 28), (42, 29), (39, 30), (40, 30), (43, 30), (44, 30), (46, 30), (50, 30), (36, 31), (37, 31), (38, 31), (39, 31), (45, 31), (50, 31), (36, 32), (37, 32), (44, 33), (51, 34), (55, 35), (42, 36), (56, 36), (39, 37), (47, 37), (38, 38), (35, 40), (43, 40), (38, 41), (43, 41), (57, 41), (38, 42), (39, 42), (41, 42), (42, 42), (43, 42), (35, 43), (38, 43), (41, 43), (42, 43), (43, 43), (38, 44), (41, 44), (42, 44), (43, 44), (58, 44), (42, 45), (43, 45), (59, 46), (38, 47), (49, 47), (49, 48), (49, 49), (62, 49), (38, 50), (44, 50), (49, 50), (44, 51), (49, 51), (39, 52), (44, 52), (48, 52), (49, 52), (39, 53), (44, 53), (48, 53), (49, 53), (66, 53), (44, 54), (48, 54), (49, 54), (66, 54), (44, 55), (47, 55), (48, 55), (49, 55), (65, 55), (38, 56), (43, 56), (47, 56), (48, 56), (49, 56), (63, 56), (64, 56), (66, 56), (38, 57), (43, 57), (44, 57), (60, 57), (61, 57), (62, 57), (63, 57), (64, 57)]
1010000000000000[(41, 27)]
1010001010001010

In [6]:
statement = ''

for color in pic:
  statement += 'else if ('
  for (x,y) in pic[color]:
      statement += ('((x == ' + str(x) + ' + x_white) && (y == ' + str(y) + ' + y_white)) || ')
  
  statement = statement[:-4] + (") oled_data = 16'b" + color + '; ')

print(statement)

else if (((x == 42 + x_white) && (y == 26 + y_white)) || ((x == 43 + x_white) && (y == 26 + y_white)) || ((x == 44 + x_white) && (y == 28 + y_white)) || ((x == 47 + x_white) && (y == 28 + y_white)) || ((x == 42 + x_white) && (y == 29 + y_white)) || ((x == 39 + x_white) && (y == 30 + y_white)) || ((x == 40 + x_white) && (y == 30 + y_white)) || ((x == 43 + x_white) && (y == 30 + y_white)) || ((x == 44 + x_white) && (y == 30 + y_white)) || ((x == 46 + x_white) && (y == 30 + y_white)) || ((x == 50 + x_white) && (y == 30 + y_white)) || ((x == 36 + x_white) && (y == 31 + y_white)) || ((x == 37 + x_white) && (y == 31 + y_white)) || ((x == 38 + x_white) && (y == 31 + y_white)) || ((x == 39 + x_white) && (y == 31 + y_white)) || ((x == 45 + x_white) && (y == 31 + y_white)) || ((x == 50 + x_white) && (y == 31 + y_white)) || ((x == 36 + x_white) && (y == 32 + y_white)) || ((x == 37 + x_white) && (y == 32 + y_white)) || ((x == 44 + x_white) && (y == 33 + y_white)) || ((x == 51 + x_white) && (y == 3